In [ ]:
import json
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [ ]:
# Load configuration
def load_config():
    """Load configuration from config.json"""
    config_path = "/dbfs/FileStore/configs/config.json"
    with open(config_path, 'r') as f:
        return json.load(f)

config = load_config()
silver_path = config["data_storage"]["silver_path"]
gold_daily_path = config["data_storage"]["gold_daily_path"]
gold_monthly_path = config["data_storage"]["gold_monthly_path"]

In [ ]:
def create_gold_layer():
    """Create gold layer tables for analytics"""
    # Read silver layer data
    silver_df = spark.read.format("delta").load(silver_path)
    
    # Create daily metrics gold table
    daily_metrics = silver_df \
        .groupBy("symbol", "date") \
        .agg(
            round(avg("open"), 2).alias("avg_open"),
            round(avg("close"), 2).alias("avg_close"),
            round(max("high"), 2).alias("daily_high"),
            round(min("low"), 2).alias("daily_low"),
            sum("volume").alias("total_volume"),
            round(avg("price_change_pct"), 2).alias("avg_price_change_pct"),
            round(avg("ma10"), 2).alias("ma10"),
            round(avg("ma30"), 2).alias("ma30"),
            round(avg("volatility_30d"), 4).alias("volatility_30d")
        ) \
        .withColumn("created_date", current_timestamp())
    
    # Create monthly summary gold table
    monthly_summary = silver_df \
        .groupBy("symbol", "year", "month") \
        .agg(
            round(avg("close"), 2).alias("monthly_avg_close"),
            round(max("high"), 2).alias("monthly_high"),
            round(min("low"), 2).alias("monthly_low"),
            sum("volume").alias("monthly_volume"),
            round(avg("price_change_pct"), 2).alias("monthly_avg_change_pct"),
            round(stddev("close"), 2).alias("price_volatility"),
            round(avg("ma30"), 2).alias("monthly_avg_ma30"),
            round(avg("volatility_30d"), 4).alias("monthly_avg_volatility")
        ) \
        .withColumn("created_date", current_timestamp())
    
    # Add month-over-month change
    windowSpec = Window.partitionBy("symbol").orderBy("year", "month")
    monthly_summary = monthly_summary \
        .withColumn("prev_month_close", lag("monthly_avg_close", 1).over(windowSpec)) \
        .withColumn("mom_change_pct", 
                   round((col("monthly_avg_close") - col("prev_month_close")) / col("prev_month_close") * 100, 2))
    
    # Add quarterly averages using a window function
    quarterly_window = Window.partitionBy("symbol").orderBy("year", "month").rowsBetween(-2, 0)
    monthly_summary = monthly_summary \
        .withColumn("quarterly_avg_close", round(avg("monthly_avg_close").over(quarterly_window), 2)) \
        .withColumn("quarterly_avg_volatility", round(avg("price_volatility").over(quarterly_window), 4))
    
    # Calculate renewable energy sector performance metrics
    # This assumes all symbols in the config are renewable energy stocks
    daily_sector_metrics = daily_metrics \
        .groupBy("date") \
        .agg(
            round(avg("avg_close"), 2).alias("sector_avg_price"),
            round(avg("avg_price_change_pct"), 2).alias("sector_avg_change_pct"),
            round(avg("volatility_30d"), 4).alias("sector_avg_volatility")
        ) \
        .withColumn("sector", lit("Renewable Energy"))
    
    # Write gold tables
    daily_metrics.write.format("delta").mode("overwrite") \
        .partitionBy("symbol") \
        .save(gold_daily_path)
    
    monthly_summary.write.format("delta").mode("overwrite") \
        .partitionBy("symbol", "year") \
        .save(gold_monthly_path)
    
    # Save sector metrics to a separate path
    sector_path = gold_daily_path + "_sector"
    daily_sector_metrics.write.format("delta").mode("overwrite") \
        .save(sector_path)
    
    print(f"Gold layer created with {daily_metrics.count()} daily records and {monthly_summary.count()} monthly records")
    print(f"Sector metrics created with {daily_sector_metrics.count()} records")
    
    return daily_metrics, monthly_summary, daily_sector_metrics


In [ ]:
# Execute gold layer creation
daily_metrics, monthly_summary, sector_metrics = create_gold_layer()
